In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import os

# Load the training and test data
train_df = pd.read_csv('/content/drive/MyDrive/1327317/training_set_pixel_size_and_HC.csv')
test_df = pd.read_csv('/content/drive/MyDrive/1327317/test_set_pixel_size.csv')

# Preprocess the images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/content/drive/MyDrive/1327317/training_set',
    x_col="filename",
    y_col="pixel size(mm)",
    target_size=(64, 64),
    batch_size=16,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/content/drive/MyDrive/1327317/test_set',
    x_col="filename",
    target_size=(64, 64),
    batch_size=16,
    class_mode=None)


# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(train_generator, epochs=50)

# Predict on the test set
test_generator.reset()
pred = model.predict(test_generator)

Found 999 validated image filenames.
Found 335 validated image filenames.
Epoch 1/50
63/63 [==============================] - 64s 320ms/step - loss: 0.0168 - mae: 0.0718
Epoch 2/50
63/63 [==============================] - 16s 257ms/step - loss: 0.0015 - mae: 0.0285
Epoch 3/50
63/63 [==============================] - 16s 250ms/step - loss: 9.2614e-04 - mae: 0.0227
Epoch 4/50
63/63 [==============================] - 16s 248ms/step - loss: 8.5939e-04 - mae: 0.0223
Epoch 5/50
63/63 [==============================] - 16s 248ms/step - loss: 6.7439e-04 - mae: 0.0196
Epoch 6/50
63/63 [==============================] - 17s 268ms/step - loss: 5.3958e-04 - mae: 0.0177
Epoch 7/50
63/63 [==============================] - 16s 249ms/step - loss: 4.3052e-04 - mae: 0.0161
Epoch 8/50
63/63 [==============================] - 17s 263ms/step - loss: 3.5956e-04 - mae: 0.0142
Epoch 9/50
63/63 [==============================] - 16s 258ms/step - loss: 3.1207e-04 - mae: 0.0135
Epoch 10/50
63/63 [===============

In [7]:
from sklearn.metrics import mean_absolute_error

# Get the true values
y_true = test_df['pixel size(mm)'].values

# Calculate MAE
mae = mean_absolute_error(y_true, pred)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 0.05571344657846689
